In [3]:
# Written by chatgpt
!pip3 install keras-tuner

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Load the MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape the images to be 28 x 28 x 1 (as they are grayscale)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Define the model builder function
def build_model(hp):
  model = keras.Sequential()

  # Tune the number of filters in the first convolutional layer
  hp_filters = hp.Int('filters', min_value=16, max_value=64, step=16)
  model.add(layers.Conv2D(hp_filters, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))

  # Tune the number of filters and kernel size in subsequent convolutional layers
  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(layers.Conv2D(hp.Int(f'filters_{i}', min_value=16, max_value=64, step=16),
                            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))

  model.add(layers.Flatten())

  # Tune the number of units in the dense layer
  hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
  model.add(layers.Dense(units=hp_units, activation='relu'))

  model.add(layers.Dense(10, activation='softmax'))
  
  # Tune the learning rate
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

# Define the Keras Tuner RandomSearch object
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_dir',
    project_name='mnist_tuner')

# Perform hyperparameter tuning
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Get the best model and evaluate it on the test set
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


Trial 10 Complete [00h 01m 21s]
val_accuracy: 0.9690999984741211

Best val_accuracy So Far: 0.9890999794006348
Total elapsed time: 00h 12m 02s
INFO:tensorflow:Oracle triggered exit
313/313 [==============================] - 1s 4ms/step - loss: 0.0380 - accuracy: 0.9891
Test accuracy: 0.9890999794006348
